# Forecasting Net Prophet

You’re a growth analyst at [MercadoLibre](http://investor.mercadolibre.com/investor-relations). With over 200 million users, MercadoLibre is the most popular e-commerce site in Latin America. You've been tasked with analyzing the company's financial and user data in clever ways to make the company grow. So, you want to find out if the ability to predict search traffic can translate into the ability to successfully trade the stock.

In [ ]:
# Install the required libraries if needed. (Remove """)
"""
from IPython.display import clear_output
try:
  !pip install pystan
  !pip install fbprophet
  !pip install hvplot
  !pip install holoviews
except:
  print("Error installing libraries")
finally:
  clear_output()
  print('Libraries successfully installed')
  """

In [ ]:
# Import the required libraries and dependencies
import pandas as pd
from pathlib import Path
import holoviews as hv
from fbprophet import Prophet
import hvplot.pandas
import datetime as dt
%matplotlib inline

In [ ]:
# Read in CSV file for Google Colab
# Upload the "google_hourly_search_trends.csv" file into Colab, then store in a Pandas DataFrame
# Set the "Date" column as the Datetime Index.
"""
from google.colab import files
uploaded = files.upload()

df_mercado_trends = pd.read_csv(
    'google_hourly_search_trends.csv',
    index_col='Date',
    parse_dates=True,
    infer_datetime_format=True
)
"""
# Read in CSV file on Jupyter Lab
df_mercado_trends = pd.read_csv(
    Path('Resources/google_hourly_search_trends.csv'),
    index_col='Date',
    parse_dates=True,
    infer_datetime_format=True
)
# Review the first and last five rows of the DataFrame
display(df_mercado_trends)


In [ ]:
# Review the data types of the DataFrame
df_mercado_trends.info()


In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Sliced the DataFrame to just the month of May 2020
df_may_2020 = df_mercado_trends.loc['2020-05']

# Used hvPlot to visualize the data for May 2020
df_may_2020.hvplot(title='Google Search Trend for MercadoLibre - May 2020')


In [ ]:
# Calculated the sum of the total search traffic for May 2020
traffic_may_2020 = df_may_2020['Search Trends'].sum()

# View the traffic_may_2020 value
print(f'MercadoLibre was searched {traffic_may_2020} times on Google in the month of May')


In [ ]:
# Calcluated the monhtly median search traffic across all months 
# Grouped the DataFrame by index-year and then index-month, chained the sum and the median functions
median_monthly_traffic = df_mercado_trends.groupby([(df_mercado_trends.index.year),(df_mercado_trends.index.month)]).sum().median()

# View the median_monthly_traffic value
print(f'The median monthly search traffic across all months from June 2016 to September 2020 was {median_monthly_traffic["Search Trends"]}')


In [ ]:
# Calculated percent change between May and the median monthly traffic from 2016-2020
pct_change_mercado_traffic = (traffic_may_2020/median_monthly_traffic['Search Trends'] - 1) * 100

# Calculated difference between May and the median monthly traffic from 2016-2020
trend_change = traffic_may_2020-median_monthly_traffic['Search Trends']

# Compared the seach traffic for the month of May 2020 to the overall monthly median value
print(f'The search traffic in May 2020 was {pct_change_mercado_traffic:.2f}% higher or had {trend_change} more searches than the median search amounts for all months from June 2016 - September 2020')


**Question:** Did the Google search traffic increase during the month that MercadoLibre released its financial results?

**Answer:** According to Google Trends, there was an incease in searches for MarcadoLibre, of about 8.55%.

In [ ]:
#Created dataframe grouped by mean per hour per weekday
df_mercado_hourly_weekday = df_mercado_trends.groupby([
     df_mercado_trends.index.hour,
     df_mercado_trends.index.weekday]).mean().unstack()['Search Trends']


In [ ]:
# Changed the column labels to days of week
df_mercado_hourly_weekday.columns= ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Grouped the hourly search data to plot the average traffic by the day of week 
df_mercado_hourly_weekday.hvplot(
    xlabel = 'Hour',
    ylabel= 'Search Traffic',
    group_label='Weekday',
    title='Average Searches for MercadoLibre by Weekday'
)

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Used hvPlot to visualize the hour of the day and day of week search traffic as a heatmap.
df_mercado_hourly_weekday.transpose().hvplot.heatmap(
    xlabel='Hour',
    height=350,
    width=550,
    title='Average Searches for MercadoLibre by Hour and Weekday'
)


**Question:** Does any day-of-week effect that you observe concentrate in just a few hours of that day?

**Answer:** Given the data provided it would seem that there is an uptick in traffic at beginning of the week, peaking Monday around the midnight, this is based on local time to this data set. There seems to be more searches in the late evenings/early mornings at the start of the week peaking Monday night then slowing decreasing as the week progresses.

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Grouped the hourly search data to plot the average traffic by the week of the year
df_mercado_week_year = df_mercado_trends.groupby([df_mercado_trends.index.week, df_mercado_trends.index.year]).mean().unstack()['Search Trends']
df_mercado_week_year.hvplot(
    xlabel= 'Week',
    ylabel= 'Search Traffic',
    group_label='Year',
    title='Average Searches for MercadoLibre by Week and Year')


**Question:** Does the search traffic tend to increase during the winter holiday period (weeks 40 through 52)?

**Answer:** Given the data on the line graph, there does seems to be a slight increase, though fluctuant, in traffic from Week 40 up until week 50. We do see a sharp downturn in that last 2week time period. 

In [ ]:
# Read in CSV file for Google Colab
# Uploaded the "mercado_stock_price.csv" file into Colab, then store in a Pandas DataFrame
"""
from google.colab import files
uploaded = files.upload()

df_mercado_trends = pd.read_csv(
    'mercado_stock_price.csv',
    index_col='date',
    parse_dates=True,
    infer_datetime_format=True
)
"""
# Read in CSV file on Jupyter Lab
df_mercado_stock = pd.read_csv(
    Path('Resources/mercado_stock_price.csv'),
    index_col='date',
    parse_dates=True,
    infer_datetime_format=True
)

# View the first and last five rows of the DataFrame
display(df_mercado_stock.head())
display(df_mercado_stock.tail())

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Used hvPlot to visualize the closing price of the df_mercado_stock DataFrame
df_mercado_stock.hvplot(title='Closing Prices for MercadoLibre Stock')


In [ ]:
# Concatenated the df_mercado_stock DataFrame with the df_mercado_trends DataFrame
mercado_stock_trends_df = pd.concat([df_mercado_trends, df_mercado_stock], axis=1).dropna()

# View the first and last five rows of the DataFrame
display(mercado_stock_trends_df.head())
display(mercado_stock_trends_df.tail())

In [ ]:
# For the combined dataframe, sliced just the first half of 2020 (2020-01 through 2020-06) 
first_half_2020 = mercado_stock_trends_df.loc['2020-01':'2020-06']

# View the first and last five rows of first_half_2020 DataFrame
display(first_half_2020.head())
display(first_half_2020.tail())

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Used hvPlot to visualize the close and Search Trends data
# Ploted each column on a separate axes
print('Search Trends compared to Closing Prices for MercadoLibre for first half of 2020:')
first_half_2020.hvplot(shared_axes=False, subplots=True).cols(1)



**Question:**  Market events emerged during the year of 2020 that many companies found difficult. But, after the initial shock to global financial markets, new customers and revenue increased for e-commerce platforms. Do both time series indicate a common trend that’s consistent with this narrative?

**Answer:** It does seem that MercadoLibre follows this narrative. Both graphs show a strong decrease at the end of February and through March. This is probably due to the global situation during that timeperiod. MercadoLibre is able to recover see growth from April thorough the end of the year. Thus following the prestated narrative.

In [ ]:
# Created a new column in the mercado_stock_trends_df DataFrame called Lagged Search Trends
# This column should shift the Search Trends information by one hour
mercado_stock_trends_df['Lagged Search Trends'] = mercado_stock_trends_df['Search Trends'].shift(1)

In [ ]:
# Created a new column in the mercado_stock_trends_df DataFrame called Stock Volatility
# This column calculates the standard deviation of the closing stock price return data over a 4 period rolling window
mercado_stock_trends_df['Stock Volatility'] = mercado_stock_trends_df['close'].rolling(window=4).std()


In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Used hvPlot to visualize the stock volatility
mercado_stock_trends_df['Stock Volatility'].hvplot()


***Solution Note:*** Note how volatility spiked, and tended to stay high, during the first half of 2020. This is a common characteristic of volatility in stock returns worldwide: high volatility days tend to be followed by yet more high volatility days. When it rains, it pours.

In [ ]:
# Created a new column in the mercado_stock_trends_df DataFrame called Hourly Stock Return
# This column calculates hourly return percentage of the closing price
mercado_stock_trends_df['Hourly Stock Return'] = mercado_stock_trends_df['close'].pct_change()


In [ ]:
# View the first and last five rows of the mercado_stock_trends_df DataFrame
display(mercado_stock_trends_df[['Hourly Stock Return']].head())
display(mercado_stock_trends_df[['Hourly Stock Return']].tail())

In [ ]:
# Constructed a correlation table of Stock Volatility, Lagged Search Trends, and Hourly Stock Return
mercado_stock_trends_df[['Stock Volatility', 'Lagged Search Trends', 'Hourly Stock Return']].corr()


**Question:** Does a predictable relationship exist between the lagged search traffic and the stock volatility or between the lagged search traffic and the stock price returns?

**Answer:** The data shows that there is a negative correlation between Lagged Search Trends and Stock Volatility. We do see a very small correlation between Lagged Search Trends and Hourly Stock Return.

In [ ]:
# Using the df_mercado_trends DataFrame, reset the index so the date information is no longer the index
mercado_prophet_df = df_mercado_trends.reset_index()
    
# Labeled the columns ds and y so that the syntax is recognized by Prophet
mercado_prophet_df.columns = ['ds', 'y']

# Dropped an NaN values from the prophet_df DataFrame
mercado_prophet_df = mercado_prophet_df.dropna()

# View the first and last five rows of the mercado_prophet_df DataFrame
display(mercado_prophet_df.head())
display(mercado_prophet_df.tail())

In [ ]:
# Called the Prophet function, stored as an object
model_mercado_trends = Prophet()


In [ ]:
# Fit the time-series model.
model_mercado_trends.fit(mercado_prophet_df)


In [ ]:
# Created a future dataframe to hold predictions
# Made the prediction go out as far as 2000 hours (approx 80 days)
future_mercado_trends = model_mercado_trends.make_future_dataframe(periods=2000, freq='H')

# View the last five rows of the future_mercado_trends DataFrame
display(future_mercado_trends.tail())


In [ ]:
# Made the predictions for the trend data using the future_mercado_trends DataFrame
forecast_mercado_trends = model_mercado_trends.predict(future_mercado_trends)

# Display the first five rows of the forecast_mercado_trends DataFrame
display(forecast_mercado_trends.head())


In [ ]:
# Ploted the Prophet predictions for the Mercado trends data
forecast_plot = model_mercado_trends.plot(forecast_mercado_trends)

**Question:**  How's the near-term forecast for the popularity of MercadoLibre?

**Answer:** The forecast shows that there will be a slight downturn in search traffic in the near-term. Though given the other metrix such as earnings and stock price I would expect this some downturn to not have a big effect.


In [ ]:
# Set the index in the forecast_mercado_trends DataFrame to the ds datetime column
forecast_mercado_trends = forecast_mercado_trends.set_index('ds')

# View the only the yhat,yhat_lower and yhat_upper columns from the DataFrame
display(forecast_mercado_trends[['yhat','yhat_lower','yhat_upper']])

***Solutions Note:*** `yhat` represents the most likely (average) forecast, whereas `yhat_lower` and `yhat_upper` represents the worst and best case prediction (based on what are known as 95% confidence intervals).

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# From the forecast_mercado_trends DataFrame, used hvPlot to visualize
# the yhat, yhat_lower, and yhat_upper columns over the last 2000 hours 
forecast_mercado_trends[['yhat','yhat_lower','yhat_upper']].hvplot(title='Prophet Forecast of MercadoLibre Searches')


In [ ]:
# Reset the index in the forecast_mercado_trends DataFrame
forecast_mercado_trends = forecast_mercado_trends.reset_index()

# Used the plot_components function to visualize the forecast results for the forecast_mercdo_trends DataFrame 
print('Time Series Analysis for MercadoLibre Search Traffic with Forcast')
figures_mercado_trends = model_mercado_trends.plot_components(forecast_mercado_trends)

**Question:** What time of day exhibits the greatest popularity?

**Answer:** Within the timezone this data is in, midnight is when it exhibits greatest popularity.

**Question:** Which day of week gets the most search traffic? 
   
**Answer:** Tuesday is the day of the week that shows the most search traffic.

**Question:** What's the lowest point for search traffic in the calendar year?

**Answer:** Starting near the end of September there is a sharp decrease and bottoms out in October.


In [ ]:
# Read in CSV file for Google Colab
# Uploaded the "mercado_daily_revenue.csv" file into Colab, then store in a Pandas DataFrame
# Sales are quoted in millions of US dollars
'''
from google.colab import files
uploaded = files.upload()

df_mercado_sales = pd.read_csv(
    'mercado_daily_recenue.csv',
    index_col='date',
    parse_dates=True,
    infer_datetime_format=True
)
'''
# Read in CSV file "mercado_daily_revenue.csv"
df_mercado_sales = pd.read_csv(
    Path('Resources/mercado_daily_revenue.csv',
         index_col='date', parse_dates=True,
         infer_datetime_format=True)
)
# Set the "date" column as the DatetimeIndex
df_mercado_sales = df_mercado_sales.set_index('date')

# Review the DataFrame
display(df_mercado_sales)

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Used hvPlot to visualize the daily sales figures 
df_mercado_sales.hvplot(title = 'Daily Sales Figures - MercadoLibre',rot=90)


In [ ]:
# Applied a Facebook Prophet model to the data.
# Reset the index so that date becomes a column in the DataFrame
mercado_sales_prophet_df = df_mercado_sales.reset_index()

# Adjusted the columns names to the Prophet syntax
mercado_sales_prophet_df.columns = ['ds','y']

# Visualize the DataFrame
mercado_sales_prophet_df.head()


In [ ]:
# Created the model
mercado_sales_prophet_model = Prophet()

# Fit the model
mercado_sales_prophet_model.fit(mercado_sales_prophet_df)


In [ ]:
# Predicted sales for 90 days (1 quarter) out into the future.
# Started by making a future dataframe
mercado_sales_prophet_future = mercado_sales_prophet_model.make_future_dataframe(periods=90)

# Display the last five rows of the future DataFrame
mercado_sales_prophet_future.tail()


In [ ]:
# Made predictions for the sales each day over the next quarter
mercado_sales_prophet_forecast = mercado_sales_prophet_model.predict(mercado_sales_prophet_future)

# Display the first 5 rows of the resulting DataFrame
mercado_sales_prophet_forecast.head()


In [ ]:
# Used the plot_components function to analyze seasonal patterns in the company's revenue
print('MercadoLibre Sales Time Series')
sales_prophet_plot = mercado_sales_prophet_model.plot_components(mercado_sales_prophet_forecast)


**Question:** For example, what are the peak revenue days? (Mondays? Fridays? Something else?)

**Answer:** The plot shows an upward trend at the beginning of the week peaking on Wednesday then declines the remainder of the week. Wednesday is the peak revenue day.

In [ ]:
#Holoviews extention to render hvPlots in Colab
hv.extension('bokeh')

# Plot the predictions for the Mercado sales
mercado_sales_prophet_forecast[['yhat_lower','yhat_upper','yhat','ds']].hvplot(
    x='ds',
    xlabel='date',
    ylabel='Sales',
    title = 'Sales Predictions - MercadoLibre (millions)'
)


In [ ]:
# Set the ds column as the DataFrame Index for the mercado_sales_prophet_forecast
mercado_sales_prophet_forecast_df = mercado_sales_prophet_forecast.set_index('ds')

# Display the first and last five rows of the DataFrame
display(mercado_sales_prophet_forecast_df.head())
display(mercado_sales_prophet_forecast_df.tail())

In [ ]:
# Produced a sales forecast for the finance division
# Created a forecast_quarter Dataframe for the period 2020-07-01 to 2020-09-30
mercado_sales_forecast_quarter = mercado_sales_prophet_forecast_df.loc['2020-07-01':'2020-09-30'][['yhat_upper','yhat_lower','yhat']]

# Updated the column names to provided best case (yhat_upper), worst case (yhat_lower), and most likely (yhat) scenarios
# to match what the finance division is looking for 
mercado_sales_forecast_quarter.columns = ['Best Case','Worst Case','Most Likely' ]

# Review the last five rows of the DataFrame
mercado_sales_forecast_quarter.tail()


In [ ]:
# Displayed the summed values for all the rows in the forecast_quarter DataFrame
print('The total values for the projected revenue for Q3 of 2020 (millions):')
display(mercado_sales_forecast_quarter.sum())

### Based on the forecast information generated above, produce a sales forecast for the finance division, giving them a number for expected total sales next quarter. Include best and worst case scenarios, to better help the finance team plan.

**Answer:** MercadoLibre is seeing a strong upward trend in sales and based on this forecast is expected to earn 970millon dollars in the 3rd quarter of 2020, with a lower end of 887million and an upper end of 1billon dollars.